In [34]:
import rasterio
from rasterio.plot import show
from os.path import basename, dirname, exists, join
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

In [4]:
work_dir = '/scratch/zacharykeskinen/'
lidar_files = glob(join(work_dir,'lidar','banner','*wgs84.tiff'))

In [71]:
lidar_dir = '/scratch/zacharykeskinen/lidar/banner/'
snow_on = '/scratch/zacharykeskinen/lidar/banner/snow_on_wgs84.tiff'
snow_off = '/scratch/zacharykeskinen/lidar/banner/snow_free_wgs84.tiff'
veg = '/scratch/zacharykeskinen/lidar/banner/snow_free_veg_wgs84.tiff'

In [7]:
with rasterio.open(snow_on) as snow_on_src:
    snow_on_arr = snow_on_src.read(1)
with rasterio.open(snow_off) as snow_off_src:
    snow_off_arr = snow_off_src.read(1)
with rasterio.open(veg) as veg_src:
    veg_arr = veg_src.read(1)
# mask out nan's (-3.8e38)
snow_on_arr_ma = np.ma.masked_less(snow_on_arr, 0)
snow_off_arr_ma = np.ma.masked_less(snow_off_arr, 0)
veg_arr_ma = np.ma.masked_less(veg_arr, 0)

get snow depths and save them

In [77]:
import rasterio
from rasterio.features import shapes
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling


with rasterio.open(snow_on) as img_to_crop, rasterio.open(snow_off) as img_ref:
    # merged dataset = set to crop (snow_on)
    # img['b2'] = extent to calculate (snow_off)
    # get the extents of the snow off as a geometry
    snow_off_arr = img_ref.read(1)
    extents, _ = next(shapes(np.zeros_like(snow_off_arr), transform=img_ref.profile['transform']))

    # crop the snow_on to the snowoff extents
    cropped_snow_on, crop_transf = mask(img_to_crop, [extents], crop=True)
    kwargs = img_to_crop.meta.copy()
    kwargs.update({
        'height': cropped_snow_on.shape[1],
        'width': cropped_snow_on.shape[2],
        'transform': crop_transf})
    snow_on_clip = snow_on.replace('.tiff','_clip.tiff')
    with rasterio.open(snow_on_clip, 'w', **kwargs) as dst_clip:
        dst_clip.write(cropped_snow_on)

        # reproject to match pixels between snow_on and snow_off
        kwargs = img_to_crop.meta.copy()
        kwargs.update({
            'crs': img_ref.crs,
            'transform': img_ref.transform,
            'width': img_ref.width,
            'height': img_ref.height
        })
        snow_on_clip_rep = snow_on_clip.replace('.tiff','_repro.tiff')
        with rasterio.open(snow_on_clip_rep, 'w', **kwargs) as dst:
            reproject(
                source=rasterio.band(dst_clip, 1),
                destination=rasterio.band(dst, 1),
                src_transform=img_to_crop.transform,
                src_crs=img_to_crop.crs,
                dst_transform=img_ref.transform,
                dst_crs=img_ref.crs,
                resampling=Resampling.nearest)

with rasterio.open(snow_on_clip_rep) as snow_on_src, rasterio.open(snow_off) as snow_off_src:
    a = snow_on_src.read(1)
    b = snow_off_src.read(1)
    c = a-b
    c_ma = np.ma.masked_less(c, -100)
    c_ma = np.ma.masked_greater(c_ma, 100)
    dst_fp = join(lidar_dir, 'snow_depth.tiff')
    kwargs = snow_off_src.meta.copy()
    kwargs.update({
    'height': c_ma.shape[0],
    'width': c_ma.shape[1]})
    with rasterio.open(dst_fp, 'w', **kwargs) as snow_depth:
        snow_depth.write(c_ma, indexes=1)

ValueError: Source shape (28451, 36459) is inconsistent with given indexes 1

Get Tree heights and save them

In [84]:
with rasterio.open(veg) as src_veg, rasterio.open(snow_off) as src_ground:
    a = src_veg.read(1)
    b = src_ground.read(1)
    c = a - b
    c_ma = np.ma.masked_less(c, -1000)
    c_ma = np.ma.masked_greater(c_ma, 1000)
    kwargs = src_ground.meta.copy()
    dst_fp = join(lidar_dir, 'veg_height.tiff')
    with rasterio.open(dst_fp, 'w', **kwargs) as veg_height:
        veg_height.write(c_ma, indexes=1)